In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [2]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

Task: Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation ignore the letter case, i.e., consider all words as lower case. Also, you can ignore all the words starting with a non-alphabetic character.

In [4]:
text = sc.textFile("pg100.txt")

In [6]:
text.take(5)

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or']

In [169]:
splitText = text.flatMap(lambda element: element.split(" ")).filter(lambda a: len(a)>0).filter(lambda b: re.match(r'[a-z]+.*',b))

In [170]:
lowerText = splitText.map(lambda element: element.lower())

In [171]:
lowerText.take(10)

['of', 'of', 'by', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone']

In [172]:
pairedLowerText = lowerText.map(lambda element: [element, 1])

In [173]:
pairedLowerText.take(10)

[['of', 1],
 ['of', 1],
 ['by', 1],
 ['ebook', 1],
 ['is', 1],
 ['for', 1],
 ['the', 1],
 ['use', 1],
 ['of', 1],
 ['anyone', 1]]

In [174]:
reducedText = pairedLowerText.reduceByKey(lambda v1, v2: v1+v2)

In [175]:
reducedText.take(20)

[('of', 15649),
 ('ebook', 9),
 ('is', 7874),
 ('use', 266),
 ('anyone', 4),
 ('anywhere', 4),
 ('at', 2227),
 ('no', 2439),
 ('restrictions', 2),
 ('whatsoever.', 3),
 ('may', 1341),
 ('it,', 529),
 ('give', 964),
 ('away', 294),
 ('re-use', 2),
 ('this', 4809),
 ('online', 4),
 ('www.gutenberg.org', 2),
 ('ebook,', 2),
 ('copyright', 21)]

In [176]:
#Start from lowerText
alphabeticText = lowerText.map(lambda element: [list(element)[0], 1])

In [177]:
alphaCount = alphabeticText.reduceByKey(lambda v1,v2: v1+v2)

In [178]:
alphaCount.take(alphaCount.count())

[('b', 34561),
 ('i', 32292),
 ('c', 23496),
 ('r', 10400),
 ('g', 14703),
 ('p', 19344),
 ('h', 50511),
 ('y', 21879),
 ('s', 52643),
 ('d', 23531),
 ('l', 22353),
 ('j', 1593),
 ('o', 34201),
 ('e', 10431),
 ('f', 28819),
 ('t', 101603),
 ('u', 7667),
 ('a', 63748),
 ('n', 21813),
 ('w', 44981),
 ('m', 46233),
 ('v', 4131),
 ('q', 1332),
 ('k', 5789),
 ('z', 53)]